In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import preprocess_dataset as ut
from scipy.interpolate import BSpline
from tfkan import DenseKAN
from tensorflow import keras
from keras_tuner import HyperModel, GridSearch, RandomSearch
from tensorflow import keras
from kerastuner.tuners import BayesianOptimization
from sklearn.model_selection import KFold

C:\Users\matte\AppData\Local\Temp\ipykernel_4248\978629149.py:10: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import BayesianOptimization


In [ ]:
x_train = np.random.uniform(low=0, high=10, size=(100000, 2))
y_train = x_train[:, 0] * x_train[:, 1]

# Creazione del modello KAN con parametri del tuning
kan = keras.models.Sequential([
    DenseKAN(2, grid_size=32, grid_range=(-2, 2)),
    DenseKAN(1, grid_size=32, grid_range=(-2, 2))
])
kan.build(input_shape=(None, 2))
kan.compile(optimizer=keras.optimizers.Adagrad(learning_rate=0.2), loss='mae')

# Training modello e salvataggio performance
kan.fit(x_train, y_train, epochs=100, batch_size=256)

In [16]:
mlp = keras.models.Sequential([
    keras.models.Dense(64, activation='relu'),
    keras.models.Dense(1),
])
mlp.build(input_shape=(None, 2))
mlp.compile(optimizer=keras.optimizers.Adagrad(learning_rate=0.2), loss='mae')
mlp.fit(x_train, y_train, epochs=100, batch_size=256)

AttributeError: module 'keras.api.models' has no attribute 'Dense'

In [10]:
t = tf.convert_to_tensor(np.array((0.3645, 0.9884)))
t = tf.reshape(t, (1, 2))
print(0.3645*0.9884)
print(kan.predict(t))
print(mlp.predict(t))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
[[0.35942167]]


In [2]:
x_train = tf.convert_to_tensor(pd.read_csv("datasets/x_train.csv").to_numpy()[:, 1:])
y_train = tf.convert_to_tensor(pd.read_csv("datasets/y_train.csv").to_numpy()[:, 1:])

x_val = tf.convert_to_tensor(pd.read_csv("datasets/x_val.csv").to_numpy()[:, 1:])
y_val = tf.convert_to_tensor(pd.read_csv("datasets/y_val.csv").to_numpy()[:, 1:])

x_test = tf.convert_to_tensor(pd.read_csv("datasets/x_test.csv").to_numpy()[:, 1:])
y_test = tf.convert_to_tensor(pd.read_csv("datasets/y_test.csv").to_numpy()[:, 1:])

print(x_train.shape, x_val.shape, x_test.shape, y_train.shape, y_test.shape, y_val.shape)

(60000, 26) (15000, 26) (20000, 26) (60000, 1) (20000, 1) (15000, 1)


In [3]:
kan_layers = [4,1]
kan_model = ut.build_model(kan_layers, x_train.shape[1], mlp=False)
kan_model.summary()
kan_history = kan_model.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_val, y_val), verbose=1)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_kan (DenseKAN)            │ (None, 4)              │           940 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_kan_1 (DenseKAN)          │ (None, 1)              │            37 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 977 (7.63 KB)

 Trainable params: 977 (7.63 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1693.6632 - mse: 1693.6632 - val_loss: 140.1668 - val_mse: 140.1668
Epoch 2/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 120.0072 - mse: 120.0072 - val_loss: 73.1464 - val_mse: 73.1464
Epoch 3/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 61.8716 - mse: 61.8716 - val_loss: 35.9863 - val_mse: 35.9863
Epoch 4/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 30.4754 - mse: 30.4754 - val_loss: 18.0199 - val_mse: 18.0199
Epoch 5/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 16.0840 - mse: 16.0840 - val_loss: 10.0723 - val_mse: 10.0723
Epoch 6/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 9.1299 - mse: 9.1299 - val_loss: 6.2660 - val_mse: 6.2660
Epoch 7/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 5.5821 - mse: 5.5821 - val_loss: 4.0047 - val_mse: 4.0047
Epoch 8/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.6440 - mse: 3.6440 - val_loss: 2.8353 - val_mse: 2.8353
Epoch 9/20
469/469 ━━━━━━━━━

In [4]:
## MLP Pura
#
#pure_mlp_layers = [64, 32, 16, 1]
#mlp_model = ut.build_model(pure_mlp_layers, x_train.shape[1], mlp=True, optimizer='rmsprop', loss="categorical_crossentropy")
#mlp_model.summary()
#
#mlp_history = mlp_model.fit(x_train, y_train, epochs=50, batch_size=128, validation_data=(x_test, y_test), verbose=1)

In [29]:
class MyHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        model = keras.Sequential()
        model.add(tf.keras.layers.Input(shape=self.input_shape))
        
        for i in range(hp.Int('num_layers', 2, 5)):
            model.add(DenseKAN(
                units=hp.Int('units_' + str(i), min_value=1, max_value=32, step=1),
                grid_size=hp.Int('grid_size_' + str(i), 3, 7),
                spline_order=hp.Int('spline_order_' + str(i), 2, 4)
            ))
        
        model.add(DenseKAN(1))

        learning_rate = hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')
        optimizer = hp.Choice('optimizer', ['adam', 'adagrad', 'rmsprop', 'adadelta'])
        
        if optimizer == 'adam':
            opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        elif optimizer == 'adagrad':
            opt = tf.keras.optimizers.Adagrad(learning_rate=learning_rate)
        elif optimizer == 'rmsprop':
            opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
        else:
            opt = tf.keras.optimizers.Adadelta(learning_rate=learning_rate)

        model.compile(optimizer=opt,
                      loss='mean_absolute_error',
                      metrics=[
                          tf.keras.metrics.MeanSquaredError(name='mse'),
                          tf.keras.metrics.MeanAbsoluteError(name='mae'),
                          tf.keras.metrics.RootMeanSquaredError(name='rmse')
                      ])
        return model


# Assumo che x_train, y_train siano già definiti
hypermodel = MyHyperModel(input_shape=(26,))

tuner = BayesianOptimization(
    hypermodel,
    objective='val_loss',
    max_trials=50,
    directory=r".\modelli_salvati",
    project_name='retina_kan'
)

tuner.search_space_summary()

# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6
)

# K-Fold Cross Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_indices, val_indices) in enumerate(kfold.split(x_train)):
    print(f"Fold {fold + 1}")
    train_indices = tf.convert_to_tensor(train_indices, dtype=tf.int32)
    val_indices = tf.convert_to_tensor(val_indices, dtype=tf.int32)
    
    x_train_fold, x_val_fold = tf.gather(x_train, train_indices), tf.gather(x_train, val_indices)
    y_train_fold, y_val_fold = tf.gather(y_train, train_indices), tf.gather(y_train, val_indices)

    tuner.search(x_train_fold, y_train_fold,
                 epochs=100,
                 validation_data=(x_val_fold, y_val_fold),
                 callbacks=[early_stopping, lr_scheduler])

tuner.results_summary()
best_model = tuner.get_best_models(num_models=1)[0]

# Valutazione del modello migliore sul set di test
test_loss, test_mse, test_mae, test_rmse = best_model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test MSE: {test_mse}")
print(f"Test MAE: {test_mae}")
print(f"Test RMSE: {test_rmse}")

# Salvataggio del modello migliore
best_model.save('best_retina_kan_model.h5')

Reloading Tuner from .\modelli_salvati\retina_kan\tuner0.json
Search space summary
Default search space size: 18
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 5, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 32, 'step': 1, 'sampling': 'linear'}
grid_size_0 (Int)
{'default': None, 'conditions': [], 'min_value': 3, 'max_value': 7, 'step': 1, 'sampling': 'linear'}
spline_order_0 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 4, 'step': 1, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 32, 'step': 1, 'sampling': 'linear'}
grid_size_1 (Int)
{'default': None, 'conditions': [], 'min_value': 3, 'max_value': 7, 'step': 1, 'sampling': 'linear'}
spline_order_1 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 4, 'step': 1, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions':

Test Loss: 0.02077632211148739
Test MSE: 0.033895332366228104
Test MAE: 0.020776323974132538
Test RMSE: 0.18410685658454895


In [14]:
# Ottieni i migliori iperparametri
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

# Ottieni e stampa i nomi e i valori dei parametri
for param_name in best_hyperparameters.values.keys():
    print(f"{param_name}: {best_hyperparameters.get(param_name)}")


num_layers: 4
units_0: 8
grid_size_0: 3
spline_order_0: 3
units_1: 21
grid_size_1: 4
spline_order_1: 4
learning_rate: 0.0050089061837515795
optimizer: adam
units_2: 32
grid_size_2: 7
spline_order_2: 2
units_3: 8
grid_size_3: 6
spline_order_3: 2
units_4: 11
grid_size_4: 5
spline_order_4: 2


In [15]:
# Ottieni i migliori iperparametri per i primi 10 modelli
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=10)

# Per ogni set di iperparametri
for i, hyperparameters in enumerate(best_hyperparameters):
    print(f"Modello {i+1}")
    # Ottieni e stampa i nomi e i valori dei parametri
    for param_name in hyperparameters.values.keys():
        print(f"{param_name}: {hyperparameters.get(param_name)}")
    print("\n")


Modello 1
num_layers: 4
units_0: 8
grid_size_0: 3
spline_order_0: 3
units_1: 21
grid_size_1: 4
spline_order_1: 4
learning_rate: 0.0050089061837515795
optimizer: adam
units_2: 32
grid_size_2: 7
spline_order_2: 2
units_3: 8
grid_size_3: 6
spline_order_3: 2
units_4: 11
grid_size_4: 5
spline_order_4: 2


Modello 2
num_layers: 5
units_0: 22
grid_size_0: 5
spline_order_0: 2
units_1: 5
grid_size_1: 4
spline_order_1: 3
learning_rate: 0.0003082442963419899
optimizer: adam
units_2: 24
grid_size_2: 5
spline_order_2: 3
units_3: 27
grid_size_3: 7
spline_order_3: 4
units_4: 13
grid_size_4: 7
spline_order_4: 2


Modello 3
num_layers: 4
units_0: 4
grid_size_0: 3
spline_order_0: 4
units_1: 23
grid_size_1: 5
spline_order_1: 3
learning_rate: 0.0015324287478969592
optimizer: adam
units_2: 26
grid_size_2: 4
spline_order_2: 4
units_3: 14
grid_size_3: 4
spline_order_3: 2
units_4: 16
grid_size_4: 3
spline_order_4: 4


Modello 4
num_layers: 4
units_0: 22
grid_size_0: 4
spline_order_0: 4
units_1: 23
grid_size_1